In [1]:
import pandas
import numpy as np
import re

df = pandas.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                       'key2' : ['one', 'two', 'one', 'two', 'one'],
                       'data1' : np.random.randn(5), 'data2' : np.random.randn(5)})
grouped = df['data1'].groupby(df['key1'])
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])
o  = [
    df, '','.mean()',
    grouped.mean(), '','df["data1"].groupby(df["key1"])',
    means, '','',
    means.unstack(), '','',
    df['data1'].groupby([states, years]).mean(), '','',
    # use a simple column name
    df.groupby('key1').mean(), '', 
    'df.groupby(["key1", "key2"]).mean()',
    df.groupby(['key1', 'key2']).mean(), '', 
    'df.groupby(["key1", "key2"]).size()',
    df.groupby(['key1', 'key2']).size(), '','',
]
o

[  key1 key2     data1     data2
 0    a  one  0.583269 -0.480127
 1    a  two -1.822070 -0.854452
 2    b  one  0.820525 -0.264516
 3    b  two  0.822191 -0.889029
 4    a  one -0.110067 -0.918224,
 '',
 '.mean()',
 key1
 a   -0.449623
 b    0.821358
 Name: data1, dtype: float64,
 '',
 'df["data1"].groupby(df["key1"])',
 key1  key2
 a     one     0.236601
       two    -1.822070
 b     one     0.820525
       two     0.822191
 Name: data1, dtype: float64,
 '',
 '',
 key2       one       two
 key1                    
 a     0.236601 -1.822070
 b     0.820525  0.822191,
 '',
 '',
 California  2005   -1.822070
             2006    0.820525
 Ohio        2005    0.702730
             2006   -0.110067
 Name: data1, dtype: float64,
 '',
 '',
          data1     data2
 key1                    
 a    -0.449623 -0.750934
 b     0.821358 -0.576772,
 '',
 'df.groupby(["key1", "key2"]).mean()',
               data1     data2
 key1 key2                    
 a    one   0.236601 -0.699176
      two  

## Iterating Over Groups

In [2]:
df

,key1,key2,data1,data2
0,a,one,0.583269,-0.480127
1,a,two,-1.822070,-0.854452
2,b,one,0.820525,-0.264516
3,b,two,0.822191,-0.889029
4,a,one,-0.110067,-0.918224


In [3]:
for name, group in df.groupby('key1'):
    print("name: "+name+", \ngroup:")
    print(group)

name: a, 
group:
  key1 key2     data1     data2
0    a  one  0.583269 -0.480127
1    a  two -1.822070 -0.854452
4    a  one -0.110067 -0.918224
name: b, 
group:
  key1 key2     data1     data2
2    b  one  0.820525 -0.264516
3    b  two  0.822191 -0.889029


In [4]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print(f'k1: {k1}, k2: {k2}, \ngroup:')
    print(group)

k1: a, k2: one, 
group:
  key1 key2     data1     data2
0    a  one  0.583269 -0.480127
4    a  one -0.110067 -0.918224
k1: a, k2: two, 
group:
  key1 key2    data1     data2
1    a  two -1.82207 -0.854452
k1: b, k2: one, 
group:
  key1 key2     data1     data2
2    b  one  0.820525 -0.264516
k1: b, k2: two, 
group:
  key1 key2     data1     data2
3    b  two  0.822191 -0.889029


In [5]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print(f"\n\nk1: {k1}, k2: {k2}")
    print(group)

one_liner = dict(list(df.groupby('key1', axis=0)))

# are equivalent
g1a = df.groupby('key1')['data1']
g1b = df['data1'].groupby(df['key1'])
# are equivalent
g2a = df.groupby('key1')[['data1']]
g2b = df[['data1']].groupby(df['key1'])

g3a = df.groupby(['key1', 'key2'])[['data2']].mean()

o = [
    'g1a', g1a,
    'g1b', g1b,
    'g2a', g2a,
    'g2b', g2b,
    'g3a', g3a
]
o



k1: a, k2: one
  key1 key2     data1     data2
0    a  one  0.583269 -0.480127
4    a  one -0.110067 -0.918224


k1: a, k2: two
  key1 key2    data1     data2
1    a  two -1.82207 -0.854452


k1: b, k2: one
  key1 key2     data1     data2
2    b  one  0.820525 -0.264516


k1: b, k2: two
  key1 key2     data1     data2
3    b  two  0.822191 -0.889029


['g1a',
 'g1b',
 'g2a',
 'g2b',
 'g3a',
               data2
 key1 key2          
 a    one  -0.699176
      two  -0.854452
 b    one  -0.264516
      two  -0.889029]

## Grouping with Dicts and Series

In [6]:
people = pandas.DataFrame(np.random.randn(5, 5), columns=['a', 'b', 'c', 'd', 'e'],
                          index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people.iloc[2:3, [2,3]] = np.nan
people

,a,b,c,d,e
Joe,-0.029188,0.687223,2.458366,-0.794209,1.407477
Steve,-0.765251,-0.093360,-0.839493,-0.926503,-0.581823
Wes,0.335183,0.041830,NaN,NaN,0.538323
Jim,0.647030,-1.306387,1.505366,1.047923,1.942102
Travis,-1.370521,0.450508,-0.306474,0.425125,-0.708397


In [7]:
mapping = {
    'a': 'red',
    'b': 'red',
    'c': 'blue',
    'd': 'blue',
    'e': 'red',
    'f' : 'orange'
}
map_series = pandas.Series(mapping)
o = [
    people.groupby(mapping, axis=1).sum(), '','',
    people.groupby(map_series, axis=1).count(), '','',
]
o

[            blue       red
 Joe     1.664157  2.065511
 Steve  -1.765996 -1.440434
 Wes     0.000000  0.915335
 Jim     2.553289  1.282745
 Travis  0.118651 -1.628410,
 '',
 '',
         blue  red
 Joe        2    3
 Steve      2    3
 Wes        0    3
 Jim        2    3
 Travis     2    3,
 '',
 '']

## Grouping with Functions

In [8]:
key_list = ['one', 'one', 'one', 'two', 'two']
o = [
    people.groupby(len).sum(), '','',
    people.groupby([len, key_list]).min(), '','',
]
o

[          a         b         c         d         e
 3  0.953025 -0.577335  3.963731  0.253715  3.887902
 5 -0.765251 -0.093360 -0.839493 -0.926503 -0.581823
 6 -1.370521  0.450508 -0.306474  0.425125 -0.708397,
 '',
 '',
               a         b         c         d         e
 3 one -0.029188  0.041830  2.458366 -0.794209  0.538323
   two  0.647030 -1.306387  1.505366  1.047923  1.942102
 5 one -0.765251 -0.093360 -0.839493 -0.926503 -0.581823
 6 two -1.370521  0.450508 -0.306474  0.425125 -0.708397,
 '',
 '']

## Grouping by Index Levels

In [9]:
columns = pandas.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                                         [1, 3, 5, 1, 3]],
                                        names=['cty', 'tenor'])
hier_df = pandas.DataFrame(np.random.randn(4, 5), columns=columns)
o = [
    '','',
    hier_df, '','',
    hier_df.groupby(level='cty', axis=1).sum()
]
o

['',
 '',
 cty          US                            JP          
 tenor         1         3         5         1         3
 0      1.745668 -0.350485  1.060154 -1.159785 -0.150891
 1     -0.891322  1.941048  1.556550 -0.869757  0.286392
 2     -0.445934 -1.363256 -1.120042 -0.030304 -0.978301
 3      1.159759 -1.468242 -0.122778 -0.266320  0.429568,
 '',
 '',
 cty        JP        US
 0   -1.310676  2.455337
 1   -0.583365  2.606276
 2   -1.008604 -2.929232
 3    0.163248 -0.431261]

# Data Aggregation

In [10]:
df

,key1,key2,data1,data2
0,a,one,0.583269,-0.480127
1,a,two,-1.822070,-0.854452
2,b,one,0.820525,-0.264516
3,b,two,0.822191,-0.889029
4,a,one,-0.110067,-0.918224


In [11]:
"""
.quantile()
.agg()
.describe()
"""

def peak_to_peak(arr):
    return arr.max() - arr.min()

grouped = df.groupby('key1')
o = [
    grouped, '','grouped.quantile(0.9)',
    grouped.quantile(0.9), '','grouped.agg(peak_to_peak)',
    grouped.agg(peak_to_peak), '', 'grouped.describe()',
    grouped.describe()
]
o

['',
 'grouped.quantile(0.9)',
          data1     data2
 key1                    
 a     0.444602 -0.554992
 b     0.822025 -0.326967,
 '',
 'grouped.agg(peak_to_peak)',
          data1     data2
 key1                    
 a     2.405340  0.438096
 b     0.001666  0.624513]

## Column-Wise and Multiple Function Application

In [ ]:
"""
.groupby().agg(tuples)
.groupby().agg(dictionary)
"""

## Returning Aggregated Data Without Row Indexes

In [12]:
# as_index=False to .groupby()
g = df.groupby(['key1', 'key2'], as_index=False)

# Apply: General split-apply-combine

In [13]:
"""

.groupby().apply(function_ref) // internally uses pandas.concat()

to pass in additional params e.g.
tips.groupby(['smoker', 'day'])
    .apply(top, n=1, column='total_bill') // 'top' is the function

can use .unstack() on a .describe()
result = tips.groupby('smoker')['tip_pct'].describe()
result.unstack('smoker')

"""

def top(dataframe, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:]

## Suppressing the Group Keys

In [ ]:
"""

Can pass group_keys=False to .groupby()
tips.groupby('smoker', group_keys=False).apply(top)

"""

## Quantile and Bucket Analysis

In [14]:
"""

- We can pass a categorical object created by .cut() "equal-length"
quartiles = pd.cut(frame.data1, 4)
grouped = frame.data2.groupby(quartiles)
grouped.apply(get_stats).unstack() // get_stats() is beneath this multi-line comment

- using .qcut() "equal-size"
grouping = pd.qcut(frame.data1, 10, labels=False)
grouped = frame.data2.groupby(grouping)
grouped.apply(get_stats).unstack()

"""

def get_stats(s_group):
    return {'min': s_group.min(), 'max': s_group.max(),
            'count': s_group.count(), 'mean': s_group.mean()}

## Example: Filling missing Values with Group specific values

In [15]:
# code

## Example: Random Sampling and Permutation

In [16]:
# code

## Example: Group Weighted Average and Correlation

In [17]:
# code

## Example: Group-Wise Linear Regression

In [18]:
# code

# Pivot Tables and Cross Tabulation
## Cross Tabulations: Crosstab

In [19]:
# code